In [69]:
import os
import numpy as np

In [8]:
path_home = os.path.expanduser('~/')
path_67p = path_home+'Documents/year1/shape_modelling/67p/'

In [13]:
# Load in apparent magnitudes (determined from phase func and shape model)
mag_file = path_67p+'67P_LCs_R.dat'
f = open(mag_file, 'r')
lines = f.readlines()
r_mags = []
for line in lines[1:]:
    data = line.split()
    r_mags.append(float(data[2]))
# r_mags contains all the r-band photometry
# if the LSST filter is r, we make no conversion
# for filters ugizy, will need to use (solar) colour indices to convert

In [26]:
# Read in output from SQL 5sigmaDepth query
# If you reset this, go and remove parenthesis manually
sql_file = path_67p+'LSST_SQL_output.txt'
f = open(sql_file, 'r')
lines = f.readlines()
filt_list = []; five_sig_depth = []
for line in lines:
    data = line.split()
    filt_list.append(str(data[1]))
    five_sig_depth.append(float(data[2]))

In [41]:
# Now want to determine uncertainties using five sigma depths in every colour filter
# solar-analog colour indices to use:
u = -25.3
g = -26.52
r = -26.93
i = -27.05
z = -27.07
y = -27.07

col_inds = [u-r,g-r,i-r,z-r,y-r]
col_inds

[1.629999999999999,
 0.41000000000000014,
 -0.120000000000001,
 -0.14000000000000057,
 -0.14000000000000057]

In [34]:
# Function to extrapolate SNR from 5sigdepth

def determine_snr(m_sig5, m2):
    delta_m = m_sig5 - m2
    snr_unknown = 5./(10.**(-delta_m/2.5))
    return snr_unknown

In [75]:
# Find whether LSST detects object in obsID
snr_list = []
for i, filt in enumerate(filt_list):
    if filt == 'r':
        snr = determine_snr(five_sig_depth[i], r_mags[i])
        print('snr=%.4f, rmag =%.4f, 5-sig depth=%.4f'%(snr, r_mags[i], five_sig_depth[i]))
    if filt == 'u':
        umag = r_mags[i]+col_inds[0]
        snr = determine_snr(five_sig_depth[i], umag)
        print('snr=%.4f, umag =%.4f, 5-sig depth=%.4f'%(snr, umag, five_sig_depth[i]))
    if filt == 'g':
        gmag = r_mags[i]+col_inds[1]
        snr = determine_snr(five_sig_depth[i], gmag)
        print('snr=%.4f, gmag =%.4f, 5-sig depth=%.4f'%(snr, gmag, five_sig_depth[i]))
    if filt == 'i':
        imag = r_mags[i]+col_inds[2]
        snr = determine_snr(five_sig_depth[i], imag)
        print('snr=%.4f, imag =%.4f, 5-sig depth=%.4f'%(snr, imag, five_sig_depth[i]))
    if filt == 'z':
        zmag = r_mags[i]+col_inds[3]
        snr = determine_snr(five_sig_depth[i], zmag)
        print('snr=%.4f, zmag =%.4f, 5-sig depth=%.4f'%(snr, zmag, five_sig_depth[i]))
    if filt == 'y':
        ymag = r_mags[i]+col_inds[4]
        snr = determine_snr(five_sig_depth[i], ymag)
        print('snr=%.4f, ymag =%.4f, 5-sig depth=%.4f'%(snr, ymag, five_sig_depth[i]))
    snr_list.append(snr)

snr=2.7827, ymag =22.3125, 5-sig depth=21.6762
snr=3.0829, ymag =22.0901, 5-sig depth=21.5651
snr=7.6688, zmag =22.2596, 5-sig depth=22.7240
snr=11.5521, zmag =22.0878, 5-sig depth=22.9970
snr=16.5478, imag =22.1612, 5-sig depth=23.4606
snr=13.6144, imag =22.1612, 5-sig depth=23.2488
snr=12.8458, zmag =22.0400, 5-sig depth=23.0645
snr=13.1648, zmag =22.0397, 5-sig depth=23.0908
snr=12.6936, zmag =22.0429, 5-sig depth=23.0544
snr=12.9730, zmag =22.0442, 5-sig depth=23.0794
snr=5.1134, ymag =22.1875, 5-sig depth=22.2119
snr=33.6683, rmag =22.2619, 5-sig depth=24.3325
snr=9.6339, ymag =21.8515, 5-sig depth=22.5636
snr=8.8930, ymag =21.8886, 5-sig depth=22.5138
snr=25.2347, imag =21.8941, 5-sig depth=23.6517
snr=5.6915, ymag =22.0649, 5-sig depth=22.2055
snr=8.7722, zmag =21.9777, 5-sig depth=22.5880
snr=8.6992, zmag =21.9694, 5-sig depth=22.5707
snr=9.7493, zmag =21.9534, 5-sig depth=22.6784
snr=9.3304, zmag =21.9523, 5-sig depth=22.6296
snr=8.8576, zmag =21.9965, 5-sig depth=22.6174
snr=

In [73]:
unc_list = 1./np.array(snr_list)

In [77]:
fout = open(path_67p+'LSST_uncertainties_from_colours.txt', 'w+')
for val in unc_list:
    fout.write(str('%.4f'%val)+'\n')
fout.close()